In [132]:
import re
import pandas as pd

## 알고리즘1

In [133]:
datasetname = 'mwoz' # metalwoz, mwoz, selfdialog, sgd
column_names = ['turn', 'dialogue', 'label']

train = pd.read_csv(f'/home/jihyeon41/research_dial_embedding/dial2vec_git/dial2vec/datasets/{datasetname}/train.tsv', sep = '\t', header=None, names=column_names)
train['turn'] = train['turn'].astype('str')
train = train.head(5)
display(train)

,turn,dialogue,label
0,10101010101010,"I need lodgings, and I'd like to stay in an ex...",0
1,1010101010101010,I'm looking for a medium priced Chinese restau...,0
2,101010101010,I would like to eat at an expensive european f...,0
3,1010101010101010,"I feel like visiting a museum, find me one ple...",0
4,1010101010101010,I'd like to find a college to visit in the cen...,0


In [117]:
dials = train['dialogue'].str.split('|').tolist()
dials[0]

["I need lodgings, and I'd like to stay in an expensive guesthouse.#Our database doesn't list an expensive guesthouse, would you like me to try something else?#I would like a 4 star hotel, in an expensive guesthouse please. Thanks!#I don't have anything in that area that meets those criteria. Can I try searching something else?#What about a moderate priced one instead?#I have 11 results. Do you prefer to stay in a particular area of town?#I don't have an area preference.#I located avalon guesthouse located north, is a 4 and moderately priced. Would you like me to book a room for how many guest, days and starting on what day?#Do you know what area that is in and whether they have internet?#They are wifi capable and in the north area, would you like more information or a booking?#Can I please have the address along with post code?#Sure, the address is 62 Gilbert Road and the postcode is cb43pd.#Thanks, that takes care of everything for me. Have a good one!#Have a great time on your trip.

In [122]:
all_qa = []

for dial in dials[0]: # 10개 중 1개의 대화씩 접근
    que = [] # question list
    ans = [] # answer list
    qa = []
    sen = re.split(r'([.!?~])', ''.join(dial))[:-1] # 문장 끝나는데 사용되는 특수기호 다른 거 있으면 추가 필요 todo
    sen = [sen[i] + sen[i+1] for i in range(0, len(sen), 2)]

    for i in range(len(sen)):
        if sen[i].endswith('?') and i + 1 < len(sen): # # '?'로 끝나고 동시에 다음 요소가 있는 경우에만 처리
            que.append(sen[i].strip('#'))
            ans.append(sen[i+1].strip('#')) # (주의) 질문에 대한 대답이 질문이기도 함 -> 이 경우 질문 리스트와 대답 리스트에 모두 포함됨
    # print(len(que)) # 확인 필요 todo
    if len(que) > 0 and len(ans) > 0: # # 질문과 대답이 적어도 한 쌍 이상인 경우에만 추가
        qa = '#'.join(["#".join(pair) for pair in zip(que, ans)])
    all_qa.append(qa)
    
all_qa = "|".join(all_qa)

In [123]:
all_qa

"Our database doesn't list an expensive guesthouse, would you like me to try something else?#I would like a 4 star hotel, in an expensive guesthouse please.# Can I try searching something else?#What about a moderate priced one instead?#What about a moderate priced one instead?#I have 11 results.# Do you prefer to stay in a particular area of town?#I don't have an area preference.# Would you like me to book a room for how many guest, days and starting on what day?#Do you know what area that is in and whether they have internet?#Do you know what area that is in and whether they have internet?#They are wifi capable and in the north area, would you like more information or a booking?#They are wifi capable and in the north area, would you like more information or a booking?#Can I please have the address along with post code?#Can I please have the address along with post code?#Sure, the address is 62 Gilbert Road and the postcode is cb43pd.| Is there anything else I can do to help you?#I wou

In [126]:
def add_qa(dialogues):
    
    all_qa = []

    for dial in dialogues: # 10개 중 1개의 대화씩 접근
        que = [] # question list
        ans = [] # answer list
        qa = []
        sen = re.split(r'([.!?~])', ''.join(dial))[:-1] # 문장 끝나는데 사용되는 특수기호 다른 거 있으면 추가 필요 todo
        sen = [sen[i] + sen[i+1] for i in range(0, len(sen), 2)]

        for i in range(len(sen)):
            if sen[i].endswith('?') and i + 1 < len(sen): # # '?'로 끝나고 동시에 다음 요소가 있는 경우에만 처리
                que.append(sen[i].strip('#'))
                ans.append(sen[i+1].strip('#')) # (주의) 질문에 대한 대답이 질문이기도 함 -> 이 경우 질문 리스트와 대답 리스트에 모두 포함됨
        # print(len(que)) # 확인 필요 todo
        if len(que) > 0 and len(ans) > 0: # # 질문과 대답이 적어도 한 쌍 이상인 경우에만 추가
            qa = '#'.join(["#".join(pair) for pair in zip(que, ans)])
        all_qa.append(qa)
        
    return "|".join(all_qa)

# dialogue 열에 대해 함수 적용
train['dial_split'] = train['dialogue'].str.split('|').tolist()
train['qa'] = train['dial_split'].apply(add_qa)
del train['dial_split']

display(train)

,turn,dialogue,label,qa
0,10101010101010,"I need lodgings, and I'd like to stay in an ex...",0,Our database doesn't list an expensive guestho...
1,1010101010101010,I'm looking for a medium priced Chinese restau...,0,Any preference on the area?#I would like it t...
2,101010101010,I would like to eat at an expensive european f...,0,I would like to eat at an expensive european f...
3,1010101010101010,"I feel like visiting a museum, find me one ple...",0,What part of town will you be in?#I don't hav...
4,1010101010101010,I'd like to find a college to visit in the cen...,0,Can you please be more specific about what yo...


In [130]:
# qa_turn 칼럼 추가

def add_qa(dialogues):
    
    all_qa = []
    all_qa_turn = []

    for dial in dialogues: # 10개 중 1개의 대화씩 접근
        que = [] # question list
        ans = [] # answer list
        qa = []
        qa_turn = []
        sen = re.split(r'([.!?~])', ''.join(dial))[:-1] # 문장 끝나는데 사용되는 특수기호 다른 거 있으면 추가 필요 todo
        sen = [sen[i] + sen[i+1] for i in range(0, len(sen), 2)]

        for i in range(len(sen)):
            if sen[i].endswith('?') and i + 1 < len(sen): # # '?'로 끝나고 동시에 다음 요소가 있는 경우에만 처리
                que.append(sen[i].strip('#'))
                ans.append(sen[i+1].strip('#')) # (주의) 질문에 대한 대답이 질문이기도 함 -> 이 경우 질문 리스트와 대답 리스트에 모두 포함됨
                qa_turn.append('10')
        # print(len(que)) # 확인 필요 todo
        if len(que) > 0 and len(ans) > 0: # # 질문과 대답이 적어도 한 쌍 이상인 경우에만 추가
            qa = '#'.join(["#".join(pair) for pair in zip(que, ans)])
            qa_turn = ''.join(qa_turn)
        all_qa.append(qa)
        all_qa_turn.append(qa_turn)
        
    return "|".join(all_qa), "|".join(all_qa_turn)

# dialogue 열에 대해 함수 적용
train['dial_split'] = train['dialogue'].str.split('|').tolist()
train['qa'], train['qa_turn'] = zip(*train['dial_split'].apply(add_qa))
del train['dial_split']

display(train)

,turn,dialogue,label,qa,qa_turn
0,10101010101010,"I need lodgings, and I'd like to stay in an ex...",0,Our database doesn't list an expensive guestho...,1010101010101010|10101010101010|10101010101010...
1,1010101010101010,I'm looking for a medium priced Chinese restau...,0,Any preference on the area?#I would like it t...,1010101010101010|1010101010|101010101010101010...
2,101010101010,I would like to eat at an expensive european f...,0,I would like to eat at an expensive european f...,1010101010|101010|101010|101010|10101010|10101...
3,1010101010101010,"I feel like visiting a museum, find me one ple...",0,What part of town will you be in?#I don't hav...,1010101010101010|1010101010101010|101010101010...
4,1010101010101010,I'd like to find a college to visit in the cen...,0,Can you please be more specific about what yo...,1010101010|10101010101010|1010101010|101010101...


In [131]:
# 확인 -> todo
print(train['dialogue'][0])
print(train['qa'][0])
print(train['qa_turn'][0])

I need lodgings, and I'd like to stay in an expensive guesthouse.#Our database doesn't list an expensive guesthouse, would you like me to try something else?#I would like a 4 star hotel, in an expensive guesthouse please. Thanks!#I don't have anything in that area that meets those criteria. Can I try searching something else?#What about a moderate priced one instead?#I have 11 results. Do you prefer to stay in a particular area of town?#I don't have an area preference.#I located avalon guesthouse located north, is a 4 and moderately priced. Would you like me to book a room for how many guest, days and starting on what day?#Do you know what area that is in and whether they have internet?#They are wifi capable and in the north area, would you like more information or a booking?#Can I please have the address along with post code?#Sure, the address is 62 Gilbert Road and the postcode is cb43pd.#Thanks, that takes care of everything for me. Have a good one!#Have a great time on your trip.|I

## 알고리즘 2

In [154]:
# 데이터 5개만 사용
datasetname = 'mwoz' # metalwoz, mwoz, selfdialog, sgd
column_names = ['turn', 'dialogue', 'label']

train = pd.read_csv(f'/home/jihyeon41/research_dial_embedding/dial2vec_git/dial2vec/datasets/{datasetname}/train.tsv', sep = '\t', header=None, names=column_names)
train['turn'] = train['turn'].astype('str')
train = train.head(5)
display(train)

,turn,dialogue,label
0,10101010101010,"I need lodgings, and I'd like to stay in an ex...",0
1,1010101010101010,I'm looking for a medium priced Chinese restau...,0
2,101010101010,I would like to eat at an expensive european f...,0
3,1010101010101010,"I feel like visiting a museum, find me one ple...",0
4,1010101010101010,I'd like to find a college to visit in the cen...,0


In [136]:
dials = train['dialogue'].str.split('|').tolist()
dials[0]

["I need lodgings, and I'd like to stay in an expensive guesthouse.#Our database doesn't list an expensive guesthouse, would you like me to try something else?#I would like a 4 star hotel, in an expensive guesthouse please. Thanks!#I don't have anything in that area that meets those criteria. Can I try searching something else?#What about a moderate priced one instead?#I have 11 results. Do you prefer to stay in a particular area of town?#I don't have an area preference.#I located avalon guesthouse located north, is a 4 and moderately priced. Would you like me to book a room for how many guest, days and starting on what day?#Do you know what area that is in and whether they have internet?#They are wifi capable and in the north area, would you like more information or a booking?#Can I please have the address along with post code?#Sure, the address is 62 Gilbert Road and the postcode is cb43pd.#Thanks, that takes care of everything for me. Have a good one!#Have a great time on your trip.

In [141]:
all_qa_turn = []

for dial in dials[0]: # 10개 중 1개의 대화씩 접근
    
    turns = dial.split("#")
    qa_turn = []
    
    for i, turn in enumerate(turns):
        if i == 0:  # 첫 번째 턴인 경우
            if '?' in turn:
                qa_turn.append('1')
            else:
                qa_turn.append('3')
        elif i == len(turns) - 1:  # 마지막 턴인 경우
            if '?' in turns[i - 1]:
                qa_turn.append('0')
            else:
                qa_turn.append('3')
        else:  # 중간에 있는 턴인 경우
            if '?' in turns[i - 1] and '?' not in turn:  # 대답 턴 (0)
                qa_turn.append('0')
            elif '?' not in turns[i - 1] and '?' in turn:  # 질문 턴 (1)
                qa_turn.append('1')
            elif '?' in turns[i - 1] and '?' in turn:  # 질문이자 대답 턴 (2)
                qa_turn.append('2')
            else:  # 질문도 대답도 아닌 턴 (3)
                qa_turn.append('3')
        
    qa_turn = ''.join(qa_turn)
    print("dial:", dial)
    print("qa_turn:", qa_turn)
    all_qa_turn.append(qa_turn)
    
all_qa_turn = "|".join(all_qa_turn)
print("all_qa_turn:", all_qa_turn)

dial: I need lodgings, and I'd like to stay in an expensive guesthouse.#Our database doesn't list an expensive guesthouse, would you like me to try something else?#I would like a 4 star hotel, in an expensive guesthouse please. Thanks!#I don't have anything in that area that meets those criteria. Can I try searching something else?#What about a moderate priced one instead?#I have 11 results. Do you prefer to stay in a particular area of town?#I don't have an area preference.#I located avalon guesthouse located north, is a 4 and moderately priced. Would you like me to book a room for how many guest, days and starting on what day?#Do you know what area that is in and whether they have internet?#They are wifi capable and in the north area, would you like more information or a booking?#Can I please have the address along with post code?#Sure, the address is 62 Gilbert Road and the postcode is cb43pd.#Thanks, that takes care of everything for me. Have a good one!#Have a great time on your t

In [174]:
# 데이터 전체 사용
datasetname = 'sgd' # mwoz, selfdialog, sgd
path = '/home/jihyeon41/research_dial_embedding/dial2vec_git/dial2vec/datasets/'
column_names = ['turn', 'dialogue', 'label']

train = pd.read_csv(f'{path}{datasetname}/train.tsv', sep = '\t', header=None, names=column_names)
train['turn'] = train['turn'].astype('str')
# train = train.head(5)
display(train)

,turn,dialogue,label
0,10101010101010,I'd like to make a hotel reservation at First ...,0
1,1010101010101010,Could you assist me in checking the weather fo...,0
2,1010101010101010101010101010,"I'm going on a road trip, so I want to find a ...",0
3,10101010101010,Can you check the weather in Kuala Lumpur on t...,0
4,101010101010101010101010,I need a bus to Anaheim for 3 people.#When is ...,0
...,...,...,...
2192,101010101010101010101010,Can you look for round trip flights to SFO for...,0
2193,101010101010101010101010101010101010101010,I'm looking for a salon.#What is the city you ...,0
2194,1010101010101010,I would like to find some attractions in Washi...,0
2195,1010101010101010,"I need to see what the weather is in Portland,...",0


In [172]:
def add_qa_turn(dialogues):
    
    all_qa_turn = []

    for dial in dialogues: # 10개 중 1개의 대화씩 접근
        turns = dial.split("#")
        qa_turn = []

        for i, turn in enumerate(turns):
            if i == 0:  # 첫 번째 턴인 경우
                if '?' in turn:
                    qa_turn.append('1')
                else:
                    qa_turn.append('3')
            elif i == len(turns) - 1:  # 마지막 턴인 경우
                if '?' in turns[i - 1]:
                    qa_turn.append('0')
                else:
                    qa_turn.append('3')
            else:  # 중간에 있는 턴인 경우
                if '?' in turns[i - 1] and '?' not in turn:  # 대답 턴 (0)
                    qa_turn.append('0')
                elif '?' not in turns[i - 1] and '?' in turn:  # 질문 턴 (1)
                    qa_turn.append('1')
                elif '?' in turns[i - 1] and '?' in turn:  # 질문이자 대답 턴 (2)
                    qa_turn.append('2')
                else:  # 질문도 대답도 아닌 턴 (3)
                    qa_turn.append('3')
            
        qa_turn = ''.join(qa_turn)
        all_qa_turn.append(qa_turn)
        
    return "|".join(all_qa_turn)

# dialogue 열에 대해 함수 적용
train['dial_split'] = train['dialogue'].str.split('|').tolist()
train['qa_turn'] = train['dial_split'].apply(add_qa_turn)
train.drop(columns=['dial_split'], inplace=True)

display(train)

,turn,dialogue,label,qa_turn
0,10101010101010,I'd like to make a hotel reservation at First ...,0,3103103103103|33333101203103|31033331203333|31...
1,1010101010101010,Could you assist me in checking the weather fo...,0,3331010333103333|1031220333103333|103101033310...
2,1010101010101010101010101010,"I'm going on a road trip, so I want to find a ...",0,3101010310101031033310103333|31010333122010310...
3,10101010101010,Can you check the weather in Kuala Lumpur on t...,0,10333101203103|10333331220103|12010101220333|1...
4,101010101010101010101010,I need a bus to Anaheim for 3 people.#When is ...,0,310103120310310331010103|310101220310310101033...
...,...,...,...,...
2192,101010101010101010101010,Can you look for round trip flights to SFO for...,0,120101033333310103103103|310122033310310103333...
2193,101010101010101010101010101010101010101010,I'm looking for a salon.#What is the city you ...,0,310331033331031201222012010122010122033333|310...
2194,1010101010101010,I would like to find some attractions in Washi...,0,3310310103333103|3333310103103103|333331010333...
2195,1010101010101010,"I need to see what the weather is in Portland,...",0,3310103103333103|3310103103333103|331012010103...


In [173]:
train.to_csv(f"{path}{datasetname}/train_qa.tsv", sep="\t", index=False, header=False)

In [169]:
# 확인
print(train['dialogue'][0])
print(train['qa_turn'][0])

I'd like to make a hotel reservation at First Class Guest House.#When do you want to arrive?#We'll arrive on October 12th.#How many nigfine. I need a cab to get there which is at 455 Grand Avenue.#Are there other passengers travelling with you?#yes it is for two and I am good with a shared ride.#To confirm you want a shared ride cab for 2 to 455 Grand Avenue.#Yes that is right. Do you know how much it will be?#It will cost $22.44 and that is the booking through with the cab on the way.#Thanks for your assistance.#Do you want help with more?#No thanks for that.#Have a good day.|I need you to check my calendar for events.#Your appointment has been made. They have a rating of 3.7 now.#Saturday this week.#Enjoy the rest of your day.#ok fine. I need a cab to get there which is at 455 Grand Avenue.#Are you interested in setting up a visit to this place?#yes it is for two and I am good with a shared ride.#Do you want to purchase tickets?#Yes that is right. Do you know how much it will be?#You

In [170]:
# 각 행의 turn과 qa_turn의 각 턴(|로 구분된) 길이가 모두 동일한지 확인하는 함수

def check_turn_length(row):
    turn_lengths = [len(turn.split("|")) for turn in row['turn'].split("#")]
    qa_turn_lengths = [len(turn.split("|")) for turn in row['qa_turn'].split("|")]

    return all(length == turn_lengths[0] for length in turn_lengths) and \
           all(length == qa_turn_lengths[0] for length in qa_turn_lengths)

train['check'] = train.apply(check_turn_length, axis=1)
display(train)

# 'check' 열에 False가 있는지 확인
if train['check'].all():
    print("코드 구현 완료!")
else:
    print("코드 다시 짜세요.")
    train[train['check']==False]



,turn,dialogue,label,qa_turn,check
0,10101010101010,I'd like to make a hotel reservation at First ...,0,3103103103103|33333101203103|31033331203333|31...,True
1,1010101010101010,Could you assist me in checking the weather fo...,0,3331010333103333|1031220333103333|103101033310...,True
2,1010101010101010101010101010,"I'm going on a road trip, so I want to find a ...",0,3101010310101031033310103333|31010333122010310...,True
3,10101010101010,Can you check the weather in Kuala Lumpur on t...,0,10333101203103|10333331220103|12010101220333|1...,True
4,101010101010101010101010,I need a bus to Anaheim for 3 people.#When is ...,0,310103120310310331010103|310101220310310101033...,True
...,...,...,...,...,...
2192,101010101010101010101010,Can you look for round trip flights to SFO for...,0,120101033333310103103103|310122033310310103333...,True
2193,101010101010101010101010101010101010101010,I'm looking for a salon.#What is the city you ...,0,310331033331031201222012010122010122033333|310...,True
2194,1010101010101010,I would like to find some attractions in Washi...,0,3310310103333103|3333310103103103|333331010333...,True
2195,1010101010101010,"I need to see what the weather is in Portland,...",0,3310103103333103|3310103103333103|331012010103...,True


코드 구현 완료!
